# Entrez Tutorial

#### Entrez (https://www.ncbi.nlm.nih.gov/Web/Search/entrezfs.html) is a data retrieval system that provides users access to NCBI’s databases such as PubMed, GenBank, GEO, and many others. You can access Entrez from a web browser to manually enter queries, or you can use Biopython’s Bio.Entrez module for programmatic access to Entrez. The latter allows you for example to search PubMed or download GenBank records from within a Python script. The full Biopython tutorial can be found at http://biopython.org/DIST/docs/tutorial/Tutorial.html

In [2]:
from Bio import Entrez
from Bio import SeqIO
Entrez.email = "maxwell.shapiro@stonybrook.edu" # let NCBI know who you are in case they want/need to contact you

### There are certain guidelines to know before you use Entrez.
1. There is a limit to the number of requests you can have per script. Without an API key, one can make at most 3 queries per second.
2. If you spam Entrez, they can and will ban you. Without providing an email, they cannot contact you with any issues.
3. For large queries, it is recommended that you use the session history feature, something we will discuss below

# *eInfo*


#### The first method we look at is eInfo. This method gives you information about the different NCBI database, such as Pubmed, Nucleotide, Genome, etc. In general, Entrez methods yield a .xml file by default:

In [3]:
handle = Entrez.einfo()
result = handle.read()
handle.close() #always close the connection after you are done.
result

'<?xml version="1.0" encoding="UTF-8" ?>\n<!DOCTYPE eInfoResult PUBLIC "-//NLM//DTD einfo 20130322//EN" "https://eutils.ncbi.nlm.nih.gov/eutils/dtd/20130322/einfo.dtd">\n<eInfoResult>\n<DbList>\n\n\t<DbName>pubmed</DbName>\n\t<DbName>protein</DbName>\n\t<DbName>nuccore</DbName>\n\t<DbName>ipg</DbName>\n\t<DbName>nucleotide</DbName>\n\t<DbName>nucgss</DbName>\n\t<DbName>nucest</DbName>\n\t<DbName>structure</DbName>\n\t<DbName>sparcle</DbName>\n\t<DbName>genome</DbName>\n\t<DbName>annotinfo</DbName>\n\t<DbName>assembly</DbName>\n\t<DbName>bioproject</DbName>\n\t<DbName>biosample</DbName>\n\t<DbName>blastdbinfo</DbName>\n\t<DbName>books</DbName>\n\t<DbName>cdd</DbName>\n\t<DbName>clinvar</DbName>\n\t<DbName>clone</DbName>\n\t<DbName>gap</DbName>\n\t<DbName>gapplus</DbName>\n\t<DbName>grasp</DbName>\n\t<DbName>dbvar</DbName>\n\t<DbName>gene</DbName>\n\t<DbName>gds</DbName>\n\t<DbName>geoprofiles</DbName>\n\t<DbName>homologene</DbName>\n\t<DbName>medgen</DbName>\n\t<DbName>mesh</DbName>\n\t

#### Entrez can take this .xml file and make it into a python object:

In [4]:
handle = Entrez.einfo()
ncbi_dbs = Entrez.read(handle) #result is a dictionary object with one key
handle.close() #never forget!
print(ncbi_dbs.keys())

dict_keys(['DbList'])


In [5]:
print(ncbi_dbs['DbList'])

['pubmed', 'protein', 'nuccore', 'ipg', 'nucleotide', 'nucgss', 'nucest', 'structure', 'sparcle', 'genome', 'annotinfo', 'assembly', 'bioproject', 'biosample', 'blastdbinfo', 'books', 'cdd', 'clinvar', 'clone', 'gap', 'gapplus', 'grasp', 'dbvar', 'gene', 'gds', 'geoprofiles', 'homologene', 'medgen', 'mesh', 'ncbisearch', 'nlmcatalog', 'omim', 'orgtrack', 'pmc', 'popset', 'probe', 'proteinclusters', 'pcassay', 'biosystems', 'pccompound', 'pcsubstance', 'seqannot', 'snp', 'sra', 'taxonomy', 'biocollections', 'unigene', 'gencoll', 'gtr']


#### The above list contains all the databses that can be queried via Entrez. We will be primarily interested with the nucleotide database.

#### While the above query yields all the databases in NCBI, we can get information on a specific database by calling the einfo method with a particular database parameter:

In [10]:
handle = Entrez.einfo(db="nucleotide")
result = Entrez.read(handle) #Also yields a dict with one key -- 'DbInfo' again.
handle.close() #last reminder = CLOSE THE HANDLE!
print(result['DbInfo'].keys())

dict_keys(['DbName', 'MenuName', 'Description', 'DbBuild', 'Count', 'LastUpdate', 'FieldList', 'LinkList'])


#### eInfo is useful also to determine which search fields we can use. For example, when we start searching for genomes, we can search by ORGN (organism), TITL (title), etc.

In [17]:
for record in result['DbInfo']['FieldList']:
    print("%(Name)s, %(FullName)s, %(Description)s" % record)

ALL, All Fields, All terms from all searchable fields
UID, UID, Unique number assigned to each sequence
FILT, Filter, Limits the records
WORD, Text Word, Free text associated with record
TITL, Title, Words in definition line
KYWD, Keyword, Nonstandardized terms provided by submitter
AUTH, Author, Author(s) of publication
JOUR, Journal, Journal abbreviation of publication
VOL, Volume, Volume number of publication
ISS, Issue, Issue number of publication
PAGE, Page Number, Page number(s) of publication
ORGN, Organism, Scientific and common names of organism, and all higher levels of taxonomy
ACCN, Accession, Accession number of sequence
PACC, Primary Accession, Does not include retired secondary accessions
GENE, Gene Name, Name of gene associated with sequence
PROT, Protein Name, Name of protein associated with sequence
ECNO, EC/RN Number, EC number for enzyme or CAS registry number
PDAT, Publication Date, Date sequence added to GenBank
MDAT, Modification Date, Date of last update
SUBS, S

# *eSearch*

#### This method is analgous to going online and searching NCBI for something specific. For example, we can query the nucleotide database online by setting the database dropbox to "Nucelotide" and searching for *Herpesviridae,* and just like we can in an advanced search, we can specify the field to look for with the FieldList given above for the Nucleotide database:

In [57]:
handle = Entrez.esearch(db="nucleotide", term="herpesviridae [ORGN]", retmax = 60000, #[ORGN] used here because we want the organism!
                        datetype = 'pdat', mindate = "2010/01/01", maxdate = "2018/12/31", idtype="acc")
result = Entrez.read(handle) #remember, we want a python object here, not .xml
handle.close()

#### The above code yields all submitted herpesvirus nucleotide sequences published between Jan. 1, 2010 - Dec. 31, 2018, and the search output is a max of 60000 sequences.

#### The esearch method parameters are:
1. `db` **(required)**: the ncbi databse you wish to connect to. See einfo code before for list of dbs

2. `term` **(required)**: The text query you want to search for. If you do einfo in a specific db, then the search terms can be retrived with the code above

3. `retstart` **(optional)**: Say you are searching for the term in the esearch code above (herpesviridae). As of this writing this script, there are >50000 entries. This option (int) tells you how many of the first entries to skip. E.g., if retstart=10, entries 11 and on will be shown

4. `retmax` **(optional)**: Default=20. Total number of unique IDs to be shown in output. So, if the above esearch retrieves >50000, the IdList will only have a retmax 20 entries

5. `rettype` **(optional)**: Default=XML. Retrieves records in XML format. Use Entrez.read(handle) to make it a python object.Other option is 'count', which only displays total # of results

6. `retmode` **(default=xml)**: Can retrieve files as JSON as well

7. `datetype` **(optional)**: `mdat`=modification date, `pdat`=publication date, `edat`=Entrez date. The following can also be used if datetype is specified:

    1. `reldate`: set to int n, where n returns only those items specified in datetype within past n days.
    2. `mindate, maxdate`: must be used together to specify date range. format by yyyy/mm/dd.
    
8. `sort` **(optional)**: each db has different options to sort

9. `field` **(optional)**: the following are equivalent; `term='herpesviridae [ORGN]'` and `term='herpesviridae', field='ORGN'`

#### So what does the query return?

In [58]:
result.keys()

dict_keys(['Count', 'RetMax', 'RetStart', 'IdList', 'TranslationSet', 'TranslationStack', 'QueryTranslation'])

In [59]:
print("Count: " + result['Count'])
print("RetMax: " + result["RetMax"])
print("IdList (first 10): ", result['IdList'][:10])
idList = result['IdList'] #will be using this later.

Count: 22209
RetMax: 22209
IdList (first 10):  ['AP019188.1', 'AP019187.1', 'AP019186.1', 'AP019185.1', 'AP019184.1', 'AP019183.1', 'AP019182.1', 'AP019181.1', 'AP019180.1', 'AP019179.1']


# *eGQuery*

#### This method returns counts for search terms. We can accomplish this via eSearch and using the right dictionary keys, or we can count records with egquery:

In [62]:
handle = Entrez.egquery(term="herpesviridae [ORGN]") #returns dictionary with two keys: 1. "Term", 2. "eGQueryResult"
result = Entrez.read(handle)
handle.close()

print(result['eGQueryResult'])

[DictElement({'DbName': 'pubmed', 'MenuName': 'PubMed', 'Count': '100905', 'Status': 'Ok'}, attributes={}), DictElement({'DbName': 'pmc', 'MenuName': 'PubMed Central', 'Count': '2534', 'Status': 'Ok'}, attributes={}), DictElement({'DbName': 'mesh', 'MenuName': 'MeSH', 'Count': '3', 'Status': 'Ok'}, attributes={}), DictElement({'DbName': 'books', 'MenuName': 'Books', 'Count': '301', 'Status': 'Ok'}, attributes={}), DictElement({'DbName': 'pubmedhealth', 'MenuName': 'PubMed Health', 'Count': '0', 'Status': 'Term or Database is not found'}, attributes={}), DictElement({'DbName': 'omim', 'MenuName': 'OMIM', 'Count': '0', 'Status': 'Term or Database is not found'}, attributes={}), DictElement({'DbName': 'ncbisearch', 'MenuName': 'Site Search', 'Count': '1', 'Status': 'Ok'}, attributes={}), DictElement({'DbName': 'nuccore', 'MenuName': 'Nucleotide', 'Count': '55070', 'Status': 'Ok'}, attributes={}), DictElement({'DbName': 'nucgss', 'MenuName': 'GSS', 'Count': '0', 'Status': 'Term or Database

#### How many results are there in each database?

In [63]:
for record in result['eGQueryResult']:
    print(record["DbName"], record["Count"])

pubmed 100905
pmc 2534
mesh 3
books 301
pubmedhealth 0
omim 0
ncbisearch 1
nuccore 55070
nucgss 0
nucest 0
protein 244439
genome 98
structure 354
taxonomy 908
snp 0
dbvar 0
gene 9382
sra 3318
biosystems 0
unigene 0
cdd 403
clone 0
popset 806
geoprofiles 0
gds 1789
homologene 0
pccompound 0
pcsubstance 22
pcassay 4621
nlmcatalog 354
probe 1103
gap 0
proteinclusters 749
bioproject 162
biosample 2944
biocollections 0


#### In the above, we get 55070 entries in nuccore (the nucleotide database), which is different than our esearch result, but recall that we imposed a date range for published sequences in our search.

# *Epost*

#### Using ePost is a good choice when you have a long list of Ids to submit. Uploading the large list to the server would require a huge URL, which sometimes will be cut off. This method bypasses this. This is also particularly useful when we want to use the history function.

In [6]:
epost = Entrez.read(Entrez.epost(db="nucleotide", id=",".join(idList[:50])))
print(epost)

DictElement({'QueryKey': '1', 'WebEnv': 'NCID_1_194505787_130.14.18.34_9001_1550508873_239251439_0MetA0_S_MegaStore'}, attributes={})


#### Note that the two keys *WebEnv* and *QueryKey* above togehther define the session history. More details about this later.

# *eSummary*

#### eSummary retrieves summaries from an ID. Can get that list using the esearch above. Can be used in conjuction with a user history, wherein query_key and WebEnv are required parameters. Optional parameters are retstart, retmax, and retmode (see esearch above).

In [23]:
handle = Entrez.esummary(db="nucleotide", id = ",".join(idList[:10])) #returns a list of dictionaries, one dict per ID
results = Entrez.read(handle)
handle.close()
print(results[0].keys()) #get list of keys from first ID in list


dict_keys(['Item', 'Id', 'Caption', 'Title', 'Extra', 'Gi', 'CreateDate', 'UpdateDate', 'Flags', 'TaxId', 'Length', 'Status', 'ReplacedBy', 'Comment', 'AccessionVersion'])


In [24]:
print("ID: %(Id)s\n Title: %(Title)s\n Taxon: %(TaxId)s\n Add. Info: %(Extra)s" % results[0])

ID: 1519305952
 Title: Human gammaherpesvirus 4 UPN99_PBMC DNA, nearly complete genome
 Taxon: 10376
 Add. Info: gi|1519305952|dbj|AP019188.1|[1519305952]


# *eFetch*

#### Allows you to retrieve records from NCBI. Required parameters are:
1. `db=database`
2. `id=id#`

#### query_key and WebEnv also required when using in conjuction with history server (see below). 
#### Optional parameters are: 
1. `retmode` (see https://www.ncbi.nlm.nih.gov/books/NBK25499/table/chapter4.T._valid_values_of__retmode_and/?report=objectonly for allowable data formats for each db)
2. `rettype` (see link for types of records to be returned)
3. `retstart`
4. `retmax`

#### Other optional parameters specific to efetch include:
1. `strand`: 1 for plus strand, 2 for minus strand
2. `seq_start`: starting with index 1, which base to first retrieve in the sequence.
3. `seq_stop`: starting with index 1, the coordinate of the last desired base to retrieve.
4. `complexity`: Content to return. 0=entire blob, 1=bioseq, 2=minimal bioseq-set, 3=minimal nuc-prot, 4=minimal pub-set

In [28]:
handle = Entrez.efetch(db="nucleotide", id = idList[-1], rettype = "gb", retmode="text")
results = handle.read() #note that we call the read method on handle instead of using Entrez.read(handle) 
                        #because we used retmode="text" and not "xml"
handle.close()
results

'LOCUS       FJ605139                1101 bp    DNA     linear   VRL 01-JAN-2010\nDEFINITION  Suid herpesvirus 1 strain 75V19 envelope glycoprotein gI gene,\n            complete cds.\nACCESSION   FJ605139\nVERSION     FJ605139.1\nKEYWORDS    .\nSOURCE      Suid alphaherpesvirus 1\n  ORGANISM  Suid alphaherpesvirus 1\n            Viruses; dsDNA viruses, no RNA stage; Herpesvirales; Herpesviridae;\n            Alphaherpesvirinae; Varicellovirus.\nREFERENCE   1  (bases 1 to 1101)\n  AUTHORS   Van Doorsselaere,J., Glorieux,S., Favoreel,H.W. and Nauwynck,H.J.\n  TITLE     Genetic variation in gE and gI proteins of Suid herpesvirus 1\n            (Pseudorabies virus) strains from Belgian isolates and association\n            with in vitro infectivity\n  JOURNAL   Unpublished\nREFERENCE   2  (bases 1 to 1101)\n  AUTHORS   Van Doorsselaere,J., Glorieux,S., Favoreel,H.W. and Nauwynck,H.J.\n  TITLE     Direct Submission\n  JOURNAL   Submitted (03-JAN-2009) HIVB, KATHO, Wilgenstraat 32, Roeselar

##### Note that it is computationally easier to just save these files locally.

#### The above is a bit of a mess. How can we easily extract qualifiers from this file? Fortunately, Biopython has a class that can easily parse such a file

# *Biopython's SeqIO with eFetch*

#### SeqIO's parse method actually does the dirty work. This method takes in two arguments: 
1. the data handle (either a file opened for reading, or downloaded data from using Entrez, etc.)
2. a lower case string -specifying  the format. Examples include "genbank" and "fasta" (full list of allowable formats here: https://biopython.org/wiki/SeqIO)

In [30]:
from Bio import SeqIO
handle = Entrez.efetch(db="nucleotide", id = idList[65:68], rettype = "gb", retmode="text")

for seq_record in SeqIO.parse(handle,"genbank"): 
    print(">",seq_record.id)
    print(seq_record.seq)
    
handle.close()

> AP019123.1
...
> AP019122.1
...
> AP019121.1
...


#### The SeqIO is an iterator object, so using a for loop will only allow you to go through the iterator once. You can use list(SeqIO.parse(handle, format)) to change the data type to a list to see all the records at once, but this invariably uses more memory. We will save the file locally, since we don't want to continuousyl query the NCBI database: 

In [39]:
import os 

if not os.path.isfile("HSV1_17.gb"):
    handle = Entrez.efetch(db="nucleotide", id = idList[idList.index("JN555585.1")], rettype = "gb", retmode="text") #This is HSV-1 strain 17
    outfile = open("HSV1_17.gb", 'w')
    outfile.write(handle.read())
    outfile.close
    handle.close()
    print("Saved!")
else:
    print("File already on computer!")
    

File already on computer!


#### Alternatively, we can use SeqIO to write out the file:

In [45]:
handle = Entrez.efetch(db="nucleotide", id = idList[idList.index("JN555585.1")], rettype = "gb", retmode="text")
record_iterator = SeqIO.parse(handle, "genbank")
first_record = next(record_iterator)
SeqIO.write(first_record, "HSV1_17.gb", "genbank")

1

#### Now we can work with this file locally. We can still open and parse this local file via SeqIO:

In [46]:
record_iterator = SeqIO.parse("HSV1_17.gb", "genbank")
first_record = next(record_iterator) # There is only one record in this particular example.
                                     # Instead of using a for loop here, we use next(iterator) function
                                     # to read the get the record
print(first_record)

ID: JN555585.1
Name: JN555585
Description: Human herpesvirus 1 strain 17, complete genome
Number of features: 284
/molecule_type=DNA
/topology=linear
/data_file_division=VRL
/date=09-MAY-2016
/accessions=['JN555585']
/sequence_version=1
/keywords=['']
/source=Human alphaherpesvirus 1 (Herpes simplex virus 1)
/organism=Human alphaherpesvirus 1
/taxonomy=['Viruses', 'dsDNA viruses, no RNA stage', 'Herpesvirales', 'Herpesviridae', 'Alphaherpesvirinae', 'Simplexvirus']
/references=[Reference(title='Evolution of sexually transmitted and sexually transmissible human herpesviruses', ...), Reference(title='Direct Submission', ...)]
/comment=The original gene nomenclature has been retained. Genes presumably
inherited from the common ancestor of alpha-, beta- and
gammaherpesviruses (core genes) and non-core genes presumably
inherited from the ancestor of alphaherpesviruses (alpha genes) are
indicated. Initiation codons are assigned with as much confidence
as is possible for each protein-coding r

### What happends when we want to get specific annotations from the genbank retrieval?

#### We can access them using the annotations attribute, and this yields a python dictionary:

In [41]:
print(first_record.annotations.keys())

dict_keys(['molecule_type', 'topology', 'data_file_division', 'date', 'accessions', 'sequence_version', 'keywords', 'source', 'organism', 'taxonomy', 'references', 'comment', 'structured_comment'])


### What about sequence features such as coding sequences, etc.?

#### We use the features attribute:

In [42]:
print(first_record.features)

[SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(152222), strand=1), type='source'), SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(9213), strand=1), type='repeat_region'), SeqFeature(FeatureLocation(BeforePosition(0), ExactPosition(7569), strand=-1), type='gene'), SeqFeature(CompoundLocation([FeatureLocation(ExactPosition(6909), ExactPosition(7569), strand=-1), FeatureLocation(BeforePosition(0), ExactPosition(4953), strand=-1)], 'join'), type='ncRNA', location_operator='join'), SeqFeature(FeatureLocation(ExactPosition(0), ExactPosition(399), strand=1), type='repeat_region'), SeqFeature(FeatureLocation(ExactPosition(97), ExactPosition(320), strand=1), type='repeat_region'), SeqFeature(FeatureLocation(ExactPosition(512), ExactPosition(1540), strand=1), type='gene'), SeqFeature(FeatureLocation(ExactPosition(512), ExactPosition(1259), strand=1), type='CDS'), SeqFeature(FeatureLocation(ExactPosition(987), ExactPosition(1040), strand=1), type='repeat_region'), SeqFeat

#### This gives you a SeqFeature object. We can iterate through each of these objects to get info on the different annotations and attributes published in this sequence. The attributes of the SeqFeature class are:
1. `.type` : yields a description of the type of feature, e.g., type can be 'repeat region' or 'CDS,' etc.
2. `.location`: the location of the feature
3. `.strand`: in double stranded sequences, this can be 1 for top strand, 01 for bottom strand
4. `.qualifiers`: a python dictionary of of additional information. For example,the key may be "Protein_ID"

In [43]:
print(first_record.features[7].qualifiers.keys())

odict_keys(['gene', 'note', 'codon_start', 'product', 'protein_id', 'translation'])


In [44]:
print(first_record.features[7].qualifiers["gene"])

['RL1']


# *Putting multiple Entrez methods to use - an example*

#### Say we want to do a query for all the ncbi sequences that are alphaherpesviruses. We begin with an esearch on Herpes Simplex Virus 1:


In [15]:
handle = Entrez.esearch(db="Taxonomy", term = "Herpes Simplex Virus 1", retmode = "xml")
result = Entrez.read(handle)
handle.close

print(result.keys())

dict_keys(['Count', 'RetMax', 'RetStart', 'IdList', 'TranslationSet', 'TranslationStack', 'QueryTranslation'])


#### The results are in the IdList key:

In [16]:
print(result['IdList'])

['10298']


#### So the taxon Id associated with herpes simplex virus 1 in 10298. Now, let us fetch the taxonomy records using efetch:

In [17]:
handle = Entrez.efetch(db="Taxonomy", id = "10298", retmode = "xml") #returns list containing one dictionary object
result = Entrez.read(handle)
handle.close

print(result[0].keys())

dict_keys(['TaxId', 'ScientificName', 'OtherNames', 'ParentTaxId', 'Rank', 'Division', 'GeneticCode', 'MitoGeneticCode', 'Lineage', 'LineageEx', 'CreateDate', 'UpdateDate', 'PubDate'])


#### So, now we can look at the 'LineageEx' (expanded lineage) key to determine how far along the taxonomy lies *alphaherpesvirinae*:

In [21]:
lineage = result[0]['LineageEx']
lineage

[DictElement({'TaxId': '10239', 'ScientificName': 'Viruses', 'Rank': 'superkingdom'}, attributes={}), DictElement({'TaxId': '35237', 'ScientificName': 'dsDNA viruses, no RNA stage', 'Rank': 'no rank'}, attributes={}), DictElement({'TaxId': '548681', 'ScientificName': 'Herpesvirales', 'Rank': 'order'}, attributes={}), DictElement({'TaxId': '10292', 'ScientificName': 'Herpesviridae', 'Rank': 'family'}, attributes={}), DictElement({'TaxId': '10293', 'ScientificName': 'Alphaherpesvirinae', 'Rank': 'subfamily'}, attributes={}), DictElement({'TaxId': '10294', 'ScientificName': 'Simplexvirus', 'Rank': 'genus'}, attributes={})]

#### We see that this yields a list with dictionary elements. We can run through this list to look for alphaherpesvirinae:

In [22]:
for lists in lineage:
    if lists['ScientificName']=='Alphaherpesvirinae':
        taxID = lists['TaxId']
taxID

'10293'

#### Great! 10293 is the taxonomy ID associated with alphaherpesviruses. Let us do a search on this taxID: 

In [27]:
handle = Entrez.esearch(db="nucleotide", term="txid10293[ORGN]", retmax = 60000, idtype="acc") #note the txid precursor in the term parameter
result = Entrez.read(handle)
handle.close()

print(len(result['IdList']))

25254


#### Now we can use efetch to retrieve these records. 

#### However, 25254 records can be daunting to retrive, and NCBI may give us an error if we try to do it all at once. Instead, we will do it in pieces.

# *Using the history and WebEnv*

#### This is useful when you want to make a series of linked queries. We will combine esearch, epost, and efetch. When you use the history, we can download records with efetch by referrring to the session, not the ids.

#### Before, we saw that searching for all herpesvirus sequences yielded ~50,000 sequences. Say we want to download all of those 50,000 sequences. We can do this starting with the esearch method, but we will include a parameter `usehistory = "y"`. Since we are using a toy example, we will query for Psittacid Herpesvirus:

In [41]:
search_handle = Entrez.esearch(db='nucleotide', term = 'Psittacid Herpesvirus 1[ORGN]', retmax=300,  usehistory = 'y', idtype = 'acc')
search_results = Entrez.read(search_handle)
search_handle.close()

print(search_results.keys())

dict_keys(['Count', 'RetMax', 'RetStart', 'QueryKey', 'WebEnv', 'IdList', 'TranslationSet', 'TranslationStack', 'QueryTranslation'])


#### We obtain two additional keys that were not seen above during the previous vanilla esearch: the `WebEnv` and `QueryKey` keys are also listed here:

In [42]:
webenv = search_results["WebEnv"]
query_key = search_results["QueryKey"]

print(webenv, query_key)

NCID_1_279026065_130.14.22.215_9001_1550592136_1811649315_0MetA0_S_MegaStore 1


#### When we use esearch, the "Count" key describes the numbers of entries in the database, while the length of the IdList determines the number of searched sequences. The discrepency will depend on the retmax parameter, as this parameter will limit the IdList, but not the count. Here, they should be equal:

In [45]:
count = int(search_results["Count"])
idlist_len = len(search_results["IdList"])
count==idlist_len

True

#### Now, using these counts, we can write a brief script to retrieve entries in batches: 

In [51]:
from urllib.error import HTTPError  #sometimes the server is buggy and will return an HTTPError between 500 <= x < 600
                                    # and we deal with this by pausing and retrying if necessary
from time import sleep

batch_size = 3
outfile = open("PSHV1.gb", 'w')

for begin in range(0, count, batch_size):
    end = min(count, begin+batch_size)
    print("Fetching records %i to %i" % (begin+1, end))
    
    attempt = 0 #will only attempt to connect to ncbi 3 times if there is any issues with the server.
                # recall that we will need to pause, since we can only query ncbi a certain number of times a second.
    while attempt < 3:
        attempt += 1
        try:
            fetch_handle = Entrez.efetch(db="nucleotide",
                                         rettype="gb", retmode="text",
                                         retstart=begin, retmax=batch_size,
                                         webenv=webenv, query_key=query_key,
                                         idtype="acc")
            attempt = 3 #once we get a successful efetch, we want to exit this while loop
        except HTTPError as err:
            if 500 <= err.code <= 599:
                print("Error %s, attempt %i to retry..." %(err, attempt))
                sleep(5)
            else:
                raise
    data = fetch_handle.read()
    fetch_handle.close()
    outfile.write(data)
    sleep(1) # don't want to overload NCBI
outfile.close()


Fetching records 1 to 3
Fetching records 4 to 6
Fetching records 7 to 9
Fetching records 10 to 12
Fetching records 13 to 15
Fetching records 16 to 18
Fetching records 19 to 21
Fetching records 22 to 24
Fetching records 25 to 27
Fetching records 28 to 30
Fetching records 31 to 33
Fetching records 34 to 36
Fetching records 37 to 39
Fetching records 40 to 42
Fetching records 43 to 45
Fetching records 46 to 48
Fetching records 49 to 51
Fetching records 52 to 54
Fetching records 55 to 57
Fetching records 58 to 60
Fetching records 61 to 63
Fetching records 64 to 66
Fetching records 67 to 69
Fetching records 70 to 72
Fetching records 73 to 75
Fetching records 76 to 78
Fetching records 79 to 81
Fetching records 82 to 84
Fetching records 85 to 87
Fetching records 88 to 90
Fetching records 91 to 93
Fetching records 94 to 96
Fetching records 97 to 99
